In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
print(os.getcwd())
from symai import *
from IPython.display import display
from examples.sql import SQL
sql = SQL()

/Users/ryang/Work/ExtensityAI/symbolicai/notebooks
Configuration file: /Users/ryang/.symai/symai.config.json


## SQL Example

The `SQL` expression is defined and passes as a global context the syntax of the SQL language. The SQL expression is then used to generate queries based on the given context. We can then use the generated queries to get data from a database.

The following query is sent to the neural computation engine and creates a query based on the given context:

In [2]:
val = None
def _fun(x):
    global val
    val = x
Symbol._metadata.input_handler = _fun

In [3]:
Symbol('I have this class').translate('German')

<class symai.symbol.Symbol at 0x176022ad0>(value=Ich habe diesen Kurs.)

In [4]:
q = sql('Select all users above the age of 30 from the Members table.'); display(q)

<class examples.sql.SQL at 0x300e42b90>(value=SQL: SELECT * 
FROM Members 
WHERE age > 30)

We can now try to further manipulate the result by asking the model to incorporate additional information, such as filtering to a specific time range:

In [5]:
res = q << 'and limit the query to the last 10 minutes'
display(res)

<class examples.sql.SQL at 0x103eebe80>(value=SQL: SELECT * 
FROM Membe ... - INTERVAL 10 MINUTE)

We can also try to remove unwanted fields from the generated query. Notice how the model tries to remove not only the given statement but attributes associated with them:

In [6]:
res -= ' AND ...'
display(res)

<class examples.sql.SQL at 0x103eebe80>(value=SQL: SELECT * 
FROM Members 
WHERE age > 30)

And we can now even convert our query to other familiar domain specific languages, such as `SQL` or `ActiveRecord`:

In [7]:
sql_res = res.convert("ActiveRecord")
display(sql_res)

<class examples.sql.SQL at 0x176023c40>(value=Members.where("age > ?", 30))

To wrap things up, we might want to go back full circle and ask the model to generate again the explanation based on the given query:

In [8]:
answer_doc = res.query("What does this query do?")
display(answer_doc)

<class examples.sql.SQL at 0x300e42e60>(value=The query selects all col ... lue greater than 30.)

Ask it in natural language to modify the query:

In [9]:
answer = res.query("How can you limit the number of results to 30 for an SQL query?")
display(answer)

<class examples.sql.SQL at 0x300e43340>(value=SQL: SELECT * 
FROM Membe ... E age > 30 
LIMIT 30)

Even translate the explanation to a different language on the fly:

In [10]:
locale = Symbol(answer_doc).translate('German')
display(locale)

<class symai.symbol.Symbol at 0x300e43610>(value=Die Abfrage wählt alle Sp ... t größer als 30 hat.)

Fixing the query on the fly if something goes wrong or the user quickly wants to adapt a query:

In [12]:
sql.adapt(context="""Explanation: Never allow SELECT *, always use LIMIT to a max of x <= 50 entries, where x is the user specified limit.""");

In [13]:
res = sql('Select all users above the age of 30 from the Members table.')
display(res)

<class examples.sql.SQL at 0x300ef12a0>(value=SQL: SELECT *
FROM Members
WHERE age > 30
LIMIT 50)

In [15]:
sql.clear()

In [16]:
res = sql('Select all users above the age of 30 from the Members table.')
display(res)

<class examples.sql.SQL at 0x302e514b0>(value=SQL: SELECT * FROM Members WHERE age > 30)